**General Description**

The following notebook contains the code to create, train, validate, and test a rainfall-runoff model using an LSTM network architecture. The notebook support running experiments in different large-sample hydrology datasets including: CAMELS-GB, CAMELS-US, CAMELS-DE. The details for each dataset can be read from a .yml file.

***Authors:***
- Eduardo Acuña Espinoza (eduardo.espinoza@kit.edu)
- Manuel Alvarez Chaves (manuel.alvarez-chaves@simtech.uni-stuttgart.de)

In [ ]:
# Import necessary packages
import datetime
import pickle
import random
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

sys.path.append("..")
# Import classes and functions from other files
from hy2dl.datasetzoo import get_dataset
from hy2dl.evaluation.metrics import nse
from hy2dl.modelzoo import get_model
from hy2dl.training.loss import nse_basin_averaged
from hy2dl.utils.config import Config
from hy2dl.utils.optimizer import Optimizer
from hy2dl.utils.utils import set_random_seed, upload_to_device

# colorblind friendly palette
color_palette = {"observed": "#377eb8", "simulated": "#4daf4a"}

Part 1. Initialize information

In [ ]:
# Path to .yml file where the experiment settings are stored. The experimet settings can also be defined manually as a
# dictionary.
path_experiment_settings = "../examples/hybrid_camels_us.yml"

# Read experiment settings
config = Config(path_experiment_settings)
config.init_experiment()
config.dump()

Part 2. Create datasets and dataloaders used to train/validate the model

In [ ]:
# Get dataset class
Dataset = get_dataset(config)

# Dataset training
config.logger.info(f"Loading training data from {config.dataset} dataset")
total_time = time.time()

training_dataset = Dataset(cfg=config, time_period="training")

training_dataset.calculate_basin_std()
training_dataset.calculate_global_statistics(save_scaler=True)
training_dataset.standardize_data(standardize_output=False)

config.logger.info(f"Number of entities with valid samples: {len(training_dataset.df_ts)}")
config.logger.info(
    f"Time required to process {len(training_dataset.df_ts)} entities: "
    f"{datetime.timedelta(seconds=int(time.time() - total_time))}"
)
config.logger.info(f"Number of valid training samples: {len(training_dataset)}\n")

# Dataloader training
train_loader = DataLoader(
    dataset=training_dataset,
    batch_size=config.batch_size_training,
    shuffle=True,
    drop_last=True,
    collate_fn=training_dataset.collate_fn,
    num_workers=config.num_workers,
)

# Print details of a loader´s sample to check that the format is correct
config.logger.info("Details training dataloader".center(60, "-"))
config.logger.info(f"Batch structure (number of batches: {len(train_loader)})")
config.logger.info(f"{'Key':^30}|{'Shape':^30}")
# Loop through the sample dictionary and print the shape of each element
for key, value in next(iter(train_loader)).items():
    if key.startswith(("x_d", "x_conceptual")):
        config.logger.info(f"{key}")
        for i, v in value.items():
            config.logger.info(f"{i:^30}|{str(v.shape):^30}")
    else:
        config.logger.info(f"{key:<30}|{str(value.shape):^30}")

config.logger.info("")  # prints a blank line

In [ ]:
# Dataset validation
config.logger.info(f"Loading validation data from {config.dataset} dataset")
total_time = time.time()

validation_dataset = Dataset(cfg=config, time_period="validation")

validation_dataset.calculate_basin_std()
validation_dataset.scaler = training_dataset.scaler
validation_dataset.standardize_data(standardize_output=False)

config.logger.info(f"Number of entities with valid samples: {len(validation_dataset.df_ts)}")
config.logger.info(
    f"Time required to process {len(validation_dataset.df_ts)} entities: "
    f"{datetime.timedelta(seconds=int(time.time() - total_time))}"
)
config.logger.info(f"Number of valid validation samples: {len(validation_dataset)}\n")

# Dataloader training
validation_loader = DataLoader(
    dataset=validation_dataset,
    batch_size=config.batch_size_training,
    shuffle=True,
    drop_last=True,
    collate_fn=validation_dataset.collate_fn,
    num_workers=config.num_workers,
)

# Print details of a loader´s sample to check that the format is correct
config.logger.info("Details validation dataloader".center(60, "-"))
config.logger.info(f"Batch structure (number of batches: {len(validation_loader)})")
config.logger.info(f"{'Key':^30}|{'Shape':^30}")
# Loop through the sample dictionary and print the shape of each element
for key, value in next(iter(validation_loader)).items():
    if key.startswith(("x_d", "x_conceptual")):
        config.logger.info(f"{key}")
        for i, v in value.items():
            config.logger.info(f"{i:^30}|{str(v.shape):^30}")
    else:
        config.logger.info(f"{key:<30}|{str(value.shape):^30}")

config.logger.info("")  # prints a blank line

Part 3. Train model

In [ ]:
# Initialize model
set_random_seed(cfg=config)
model = get_model(config).to(config.device)

# Initialize optimizer
optimizer = Optimizer(cfg=config, model=model)

# Training report structure
config.logger.info("Training model".center(60, "-"))
config.logger.info(f"{'':^16}|{'Trainining':^21}|{'Validation':^21}|")
config.logger.info(f"{'Epoch':^5}|{'LR':^10}|{'Loss':^10}|{'Time':^10}|{'Metric':^10}|{'Time':^10}|")

total_time = time.time()
# Loop through epochs
for epoch in range(1, config.epochs + 1):
    train_time = time.time()
    loss_evol = []
    # Training -------------------------------------------------------------------------------------------------------
    model.train()
    # Loop through the different batches in the training dataset
    iterator = tqdm(
        train_loader, desc=f"Epoch {epoch}/{config.epochs}. Training", unit="batches", ascii=True, leave=False
    )

    for idx, sample in enumerate(iterator):
        # reach maximum iterations per epoch
        if config.max_updates_per_epoch is not None and idx >= config.max_updates_per_epoch:
            break

        sample = upload_to_device(sample, config.device)  # upload tensors to device
        optimizer.optimizer.zero_grad()  # sets gradients to zero

        # Forward pass of the model
        pred = model(sample)
        # Calcuate loss
        loss = nse_basin_averaged(
            y_sim=pred["y_hat"],
            y_obs=sample["y_obs"],
            per_basin_target_std=sample["std_basin"]
        )

        # Backpropagation (calculate gradients)
        loss.backward()

        # Update model parameters (e.g, weights and biases)
        optimizer.clip_grad_and_step(epoch, idx)

        # Keep track of the loss per batch
        loss_evol.append(loss.item())
        iterator.set_postfix({"loss": f"{np.mean(loss_evol):.3f}"})

        # remove elements from cuda to free memory
        del sample, pred
        torch.cuda.empty_cache()

    # training report
    report = (
        f"{epoch:^5}|"
        f"{optimizer.optimizer.param_groups[0]['lr']:^10.5f}|"
        f"{np.mean(loss_evol):^10.3f}|"
        f"{str(datetime.timedelta(seconds=int(time.time() - train_time))):^10}|"
    )

    # Validation -----------------------------------------------------------------------------------------------------
    if epoch % config.validate_every == 0:
        val_time = time.time()
        loss_evol = []
        model.eval()
        with torch.no_grad():
            iterator = tqdm(
                validation_loader,
                desc=f"Epoch {epoch}/{config.epochs}. Validation",
                unit="batches",
                ascii=True,
                leave=False,
            )

            for idx, sample in enumerate(iterator):
                # reach maximum iterations per epoch
                if config.max_updates_per_epoch is not None and idx >= config.max_updates_per_epoch:
                    break

                sample = upload_to_device(sample, config.device)
                pred = model(sample)
                loss = nse_basin_averaged(
                    y_sim=pred["y_hat"], y_obs=sample["y_obs"], per_basin_target_std=sample["std_basin"]
                )

                loss_evol.append(loss.item())
                del sample, pred
                torch.cuda.empty_cache()

            # average loss validation
            report += f"{np.mean(loss_evol):^10.3f}|{str(datetime.timedelta(seconds=int(time.time() - val_time))):^10}|"

    # No validation
    else:
        report += f"{'':^10}|{'':^10}|"

    # Print report and save model
    config.logger.info(report)
    torch.save(model.state_dict(), config.path_save_folder / "model" / f"model_epoch_{epoch}")
    # modify learning rate
    optimizer.update_optimizer_lr(epoch=epoch)

# print total training time
config.logger.info(f"Total training time: {datetime.timedelta(seconds=int(time.time() - total_time))} \n")

Part 4. Test model

In [ ]:
# In case I already trained an LSTM I can re-construct the model. I just need to define the epoch for which I want to
# re-construct the model
# model = get_model(config).to(config.device)
# model.load_state_dict(torch.load(config.path_save_folder / "model" / "model_epoch_20", map_location=config.device))

In [ ]:
# Read previously generated scaler
with open(config.path_save_folder / "scaler.pickle", "rb") as file:
    scaler = pickle.load(file)

In [ ]:
# When we run our evalation period (validation or testing), we want to differentiate between basins. Therefore, each
# batch entity will contain the whole time period (validation or testing) of a specific basin. For this, we will modify
# seq_length and the predict_last_n.
warmup_start_date = pd.to_datetime(config.testing_period[0], format="%Y-%m-%d") - pd.DateOffset(
    config.seq_length - config.predict_last_n
)

evaluation_seq_length = (pd.to_datetime(config.testing_period[1], format="%Y-%m-%d") - warmup_start_date).days + 1
evaluation_predict_last_n = evaluation_seq_length - config.predict_last_n

# Update variables in config (this is just for the testing part. One need to change it back, or read the original config
# in case training settings are required again)
config.seq_length = evaluation_seq_length
config.predict_last_n = evaluation_predict_last_n

In [ ]:
# Dataset validation
config.logger.info(f"Loading testing data from {config.dataset} dataset")
total_time = time.time()

testing_dataset = Dataset(cfg=config, time_period="testing", check_NaN=False)

testing_dataset.scaler = scaler
testing_dataset.standardize_data(standardize_output=False)

config.logger.info(f"Number of entities with valid samples: {len(testing_dataset.df_ts)}")
config.logger.info(
    "Time required to process {} entities: {}".format(
        len(testing_dataset.df_ts), datetime.timedelta(seconds=int(time.time() - total_time))
    )
)
config.logger.info(f"Number of valid testing samples: {len(testing_dataset)}\n")

# Dataloader training
test_loader = DataLoader(
    dataset=testing_dataset,
    batch_size=config.batch_size_evaluation,
    shuffle=False,
    drop_last=False,
    collate_fn=testing_dataset.collate_fn,
    num_workers=config.num_workers,
)

# Print details of a loader´s sample to check that the format is correct
config.logger.info("Details testing dataloader".center(60, "-"))
config.logger.info(f"Batch structure (number of batches: {len(test_loader)})")
config.logger.info(f"{'Key':^30}|{'Shape':^30}")
# Loop through the sample dictionary and print the shape of each element
for key, value in next(iter(test_loader)).items():
    if key.startswith(("x_d", "x_conceptual")):
        config.logger.info(f"{key}")
        for i, v in value.items():
            config.logger.info(f"{i:^30}|{str(v.shape):^30}")
    else:
        config.logger.info(f"{key:<30}|{str(value.shape):^30}")

config.logger.info("")  # prints a blank line

In [ ]:
# Testing ----------------------------------------------------------------------
config.logger.info("Testing model".center(60, "-"))
total_time = time.time()

model.eval()
with torch.no_grad():
    test_results = {}
    iterator = tqdm(test_loader, desc="Testing", unit="basins", ascii=True)
    for sample in iterator:
        sample = upload_to_device(sample, config.device)
        pred = model(sample)

        # join results in a dataframe and store them in a dictionary (is easier to plot later)
        for i in range(pred["y_hat"].shape[0]):
            df_discharge = pd.DataFrame(
                data={
                    "y_obs": sample["y_obs"][i, :, 0].flatten().cpu().detach(),
                    "y_sim": pred["y_hat"][i, :, 0].flatten().cpu().detach(),
                },
                index=pd.to_datetime(sample["date"][i, :].flatten()),
            )

            # extract internal_state (buckets) information
            internal_states = {
                key: value[i, :, :].squeeze(0).cpu().detach() for key, value in pred["internal_states"].items()
            }

            # extract parameter  information
            parameters = {key: value[i, :, :].squeeze(0).cpu().detach() for key, value in pred["parameters"].items()}

            test_results[sample["basin"][i]] = {
                "discharges": df_discharge,
                "internal_states": internal_states,
                "parameters": parameters,
            }

        # remove from cuda
        del sample, pred
        torch.cuda.empty_cache()

# Save results as a pickle file
with open(config.path_save_folder / "test_results.pickle", "wb") as f:
    pickle.dump(test_results, f)

config.logger.info(f"Total testing time: {datetime.timedelta(seconds=int(time.time() - total_time))}")

Part 5. Initial analysis

In [ ]:
discharges = {key: value["discharges"] for key, value in test_results.items()}
loss_testing = nse(df_results=discharges, average=False)
df_NSE = pd.DataFrame(data={"basin_id": list(test_results.keys()), "NSE": np.round(loss_testing, 3)})
df_NSE = df_NSE.set_index("basin_id")

# Plot the histogram
plt.hist(df_NSE["NSE"], bins=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])

# Add NSE statistics to the plot
plt.text(
    0.01,
    0.8,
    (
        f"Mean: {'%.2f' % df_NSE['NSE'].mean():>7}\n"
        f"Median: {'%.2f' % df_NSE['NSE'].median():>0}\n"
        f"Max: {'%.2f' % df_NSE['NSE'].max():>9}\n"
        f"Min: {'%.2f' % df_NSE['NSE'].min():>10}"
    ),
    transform=plt.gca().transAxes,
    bbox=dict(facecolor="white", alpha=0.5),
)

# Format plot
plt.rcParams["figure.figsize"] = (20, 5)
plt.xlabel("NSE", fontsize=12, fontweight="bold")
plt.ylabel("Frequency", fontsize=12, fontweight="bold")
plt.title("NSE Histogram", fontsize=16, fontweight="bold")
plt.show()

In [ ]:
# Plot simulated and observed discharges
basin_to_analyze = random.sample(list(test_results.keys()), 1)[0]

plt.figure(figsize=(20, 7))
plt.plot(test_results[basin_to_analyze]["discharges"]["y_obs"], label="observed", color=color_palette["observed"])
plt.plot(
    test_results[basin_to_analyze]["discharges"]["y_sim"],
    label="simulated",
    alpha=0.5,
    color=color_palette["simulated"],
)

# Format plot
plt.xlabel("Date", fontsize=12, fontweight="bold")
plt.ylabel("Discharge [mm/d]", fontsize=12, fontweight="bold")
plt.title("Modeling results", fontsize=16, fontweight="bold")
plt.tick_params(axis="both", which="major", labelsize=12)
plt.legend(loc="upper right", fontsize=12)
plt.show()

In [ ]:
# Plot states
state_of_interest = random.sample(list(test_results[basin_to_analyze]["internal_states"].keys()), 1)[0]

states = test_results[basin_to_analyze]["internal_states"][state_of_interest]

for i in range(states.shape[1]):
    plt.plot(
        test_results[basin_to_analyze]["discharges"]["y_obs"].index,
        states[:, i],
        label=state_of_interest + "_" + str(i + 1),
    )

# Adding labels and legend
plt.xlabel("Date", fontsize=12, fontweight="bold")
plt.ylabel(f"{state_of_interest}", fontsize=12, fontweight="bold")
plt.title(f"Time evolution of {state_of_interest}", fontsize=16, fontweight="bold")
plt.tick_params(axis="both", which="major", labelsize=12)
plt.legend(loc="upper right", fontsize=12)
plt.show()

In [ ]:
# Plot parameters
parameter_of_interest = random.sample(list(test_results[basin_to_analyze]["parameters"].keys()), 1)[0]

parameters = test_results[basin_to_analyze]["parameters"][parameter_of_interest]
for i in range(parameters.shape[1]):
    plt.plot(
        test_results[basin_to_analyze]["discharges"]["y_obs"].index,
        parameters[:, i],
        label=parameter_of_interest + "_" + str(i + 1),
        alpha=0.8,
    )

# Adding labels and legend
plt.xlabel("Date", fontsize=12, fontweight="bold")
plt.ylabel(f"{parameter_of_interest}", fontsize=12, fontweight="bold")
plt.title(f"Time evolution of {parameter_of_interest}", fontsize=16, fontweight="bold")
plt.tick_params(axis="both", which="major", labelsize=12)
plt.legend(loc="upper right", fontsize=12)
plt.show()